In [2]:
import os
import ccxt
# import config
import schedule
import pandas as pd
pd.set_option('display.max_rows', None)
from datetime import datetime
import time
import pytz
import json
import numpy as np
# import warnings
# warnings.filterwarnings('ignore')
import sys
# sys.path.insert(1, './strategy')
from importlib import reload  
import pyperclip
from math import floor

from dotenv import load_dotenv
load_dotenv()

True

In [132]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint

client = MongoClient()
db=client.pkbot

In [2]:
!. ./load-creds.sh

In [133]:
exchanges = {}
config = False
with open ('config.json') as js: 
        config = json.load(js)

for exkey, exconf in config['exchanges'].items():
        if exconf['enabled'] and not exkey in exchanges:
                excred = exconf['cred']
                for crk, crv in excred.items():
                        if crv[0] == "$" : 
                                excred[crk] = os.getenv(crv[1:])

                exchanges[exkey] = getattr(ccxt, exkey) (excred)

In [48]:
exchange = exchanges['kucoin']
# Working samples: 

# exchange.create_order('BTC/USDT', 'market', 'sell', 0.001)
# exchange.create_order('BTC/USDT', 'market', 'buy', 0.0009)
# exchange.create_order('BTC/USDT', 'limit', 'sell', 0.0002, 100000)
# exchange.create_order('BTC/USDT', 'limit', 'buy', 0.001, 10000)

# exchange.create_order('BTC/USDT', 'limit', 'sell', 0.005, 43900, {'stopPrice': 44000,'stop': 'loss'})
# exchange.fetch_open_orders('BTC/USDT')[0]['id']
# pyperclip.copy(exchange.fetch_open_orders('BTC/USDT')[0]['id'])
exchange.cancel_order('613e244b5ff2e50006a2c248')
exchange.fetch_open_orders('BTC/USDT')

[]

In [63]:
exchange = exchanges['ftx']
# Working samples: 

# exchange.create_order('BTC/USDT', 'market', 'sell', 0.001)
# exchange.create_order('BTC/USDT', 'market', 'buy', 0.0001)
# exchange.create_order('BTC/USDT', 'limit', 'sell', 0.0002, 100000)
# exchange.create_order('BTC/USDT', 'limit', 'buy', 0.001, 10000)
# exchange.create_order('BTC/USDT', 'stop', 'sell', 0.0005, 43900, {'stopPrice': 44000,'stop': 'stopLimit'})

# exchange.fetch_open_orders('BTC/USDT')[0]['id']
# pyperclip.copy(exchange.fetch_open_orders('BTC/USDT')[0]['id'])
# exchange.cancel_order('78771009096')
# exchange.cancel_order(exchange.fetch_open_orders('BTC/USDT')[0]['id'])
# exchange.fetch_open_orders('BTC/USDT')

In [ ]:
exchange = exchanges['ftx']
# exchange.create_order('BTC/USDT', 'market', 'sell', 0.001)
exchange.create_order('SOL/BTC', 'market', 'buy', 0.0001)
# exchange.create_order('BTC/USDT', 'limit', 'sell', 0.0002, 100000)
# exchange.create_order('BTC/USDT', 'limit', 'buy', 0.001, 10000)
# exchange.create_order('BTC/USDT', 'stop', 'sell', 0.0005, 43900, {'stopPrice': 44000,'stop': 'stopLimit'})

# exchange.fetch_open_orders('BTC/USDT')[0]['id']
# pyperclip.copy(exchange.fetch_open_orders('BTC/USDT')[0]['id'])
# exchange.cancel_order('78771009096')
# exchange.cancel_order(exchange.fetch_open_orders('BTC/USDT')[0]['id'])
# exchange.fetch_open_orders('BTC/USDT')

In [33]:
exchange = exchanges['binance']

# exchange.create_order('ALGO/USDT', 'market', 'sell', 40)
# exchange.create_order('BTC/USDT', 'market', 'buy', 0.0009)
# exchange.create_order('BTC/USDT', 'limit', 'sell', 0.0002, 100000)
# exchange.create_order('BTC/USDT', 'limit', 'buy', 0.001, 10000)
# exchange.create_order('BTC/USDT', 'STOP_LOSS_LIMIT', 'sell', 0.01, 20000, {'stopPrice': 25000,'type': 'stopLimit'})

# exchange.create_order('BTC/USDT', 'limit', 'sell', 0.005, 43900, {'stopPrice': 44000,'stop': 'loss'})
# exchange.fetch_open_orders('BTC/USDT')[0]['id'] 
# pyperclip.copy(exchange.fetch_open_orders('BTC/USDT')[0]['id'])
# exchange.cancel_order('7509903700', symbol='BTC/USDT')
# exchange.fetch_open_orders('ALGO/USDT')

exch_balances = exchange.fetch_balance ()
exch_balances['free']


In [3]:
exchange = getattr(ccxt, 'binance')({'apikey': os.getenv('BINANCE_API_KEY'), 'secret': os.getenv('BINANCE_API_SECRET')})


In [124]:
# Cancel open orders
exchange = exchanges['binance']

# exchange.create_order('ALGO/USDT', 'limit', 'buy', 100, 1.5)

# for order in exchange.fetch_open_orders('ALGO/USDT'):
#   exchange.cancel_order(order['id'], symbol=order['symbol'])

exchange.fetch_open_orders('BTC/USDT')

[]

In [134]:
def cancelOrders(symbol):
  for order in exchange.fetch_open_orders(symbol):
    exchange.cancel_order(order['id'], symbol=order['symbol'])

In [127]:
def sellMarket__priceAbove(exch, symbol, triggerPrice, amount):
  db.xqueue.insert_one({
    "description": "sellMarket__priceAbove",
    "createdTime" : datetime.now(),
    "status": "new",
    "q": [
      {
        "qid": 0,
        "symbol": symbol,
        "status": "new",
        "trigger": f"ticker['bid'] >= {triggerPrice}",
        "exchange": exch,
        "method": f"cancelOrders('{exch}', '{symbol}')"
      },
      {
        "qid": 1,
        "symbol": symbol,
        "status": "new",
        "trigger": "'done' == qx['q'][0]['status']",
        "exchange": exch,
        "method": f"exchange.create_order('{symbol}', 'market', 'sell', {amount})"
      }
    ]
  })  

In [128]:
def buyMarket__stoporderMarket(exch, symbol, triggerPrice, amount, stopPrice):
  db.xqueue.insert_one({
    "description": "buyMarket__stoporderMarket",
    "createdTime" : datetime.now(),
    "status": "new",
    "q": [
      {
        "qid": 0,
        "symbol": symbol,
        "status": "new",
        "trigger": f"ticker['ask'] < {triggerPrice}",
        "exchange": exch,
        "method": f"exchange.create_market_buy_order('{symbol}', {amount})"
      },
      {
        "qid": 1,
        "symbol": symbol,
        "status": "new",
        "trigger": "'done' == qx['q'][0]['status']",
        "exchange": exch,
        "method": f"exchange.create_order('{symbol}', 'STOP_LOSS_LIMIT', 'sell', {amount * 0.998}, {stopPrice * 0.95}, {{'stopPrice': {stopPrice}}})"
      }
    ]
  })
  

In [126]:
free_balances = exchange.fetch_balance ()['free'] 
base_available = free_balances['USDT'] 

exch, symbol = 'binance', 'ALGO/USDT'
 
triggerPrice = 1.9790
stopPrice = 1.9688
exitPrice = 1

amount = floor(base_available / triggerPrice * 0.998)

buyMarket__stoporderMarket(exch, symbol, triggerPrice, amount, stopPrice)

In [120]:
free_balances = exchange.fetch_balance ()['free'] 
base_available = free_balances['USDT'] / 100

# base_available = 100

exch, symbol = 'binance', 'ALGO/USDT'


In [135]:

exch, symbol = 'binance', 'ALGO/USDT'
 
triggerPrice = 1.98
stopPrice = 1.9380
amount = floor(base_available / triggerPrice * 0.999)
# amount

26614

In [119]:
# free_balances = exchange.fetch_balance ()['free'] 
# base_available = free_balances['USDT'] / 100

base_available = 100

exch, symbol = 'binance', 'ALGO/USDT'
 
triggerPrice = 1.98
stopPrice = 1.9380
amount = floor(base_available / triggerPrice * 0.999)

# buyMarket__stoporderMarket(exch, symbol, amount, triggerPrice, stopPrice)

# sellMarket__priceAbove(exch, symbol,  0.001, 44000)


# exchange.create_order(symbol, 'market', 'sell', amount)
# exchange.create_order(symbol, 'market', 'buy', 0.0009)
# exchange.create_order(symbol, 'limit', 'sell', 0.0002, 100000)
# exchange.create_order(symbol, 'limit', 'buy', 0.001, 10000)
# exchange.create_order(symbol', 'STOP_LOSS_LIMIT', 'sell', 0.01, 20000, {'stopPrice': 25000,'type': 'stopLimit'})

# exchange.create_order(symbol, 'limit', 'sell', 0.005, 43900, {'stopPrice': 44000,'stop': 'loss'})
# exchange.fetch_open_orders(symbol)[0]['id'] 
# pyperclip.copy(exchange.fetch_open_orders(symbol)[0]['id'])
# exchange.cancel_order('7509903700', symbol=symbol)
# exchange.fetch_open_orders(symbol')

# exch_balances = exchange.fetch_balance ()
# exch_balances['free']


In [106]:
free_balances = exchange.fetch_balance ()['free']
base_available = free_balances['USDT'] 
base_available

50761.90258599

In [97]:
floor(3.14)

NameError: name 'floor' is not defined

In [110]:
datetime.now()

AttributeError: 'datetime.datetime' object has no attribute 'to_s'

In [130]:
1.765 * 0.98

1.7296999999999998